# Introduction to NeoRV32OnPynq


## Aims

* Introduce the compiler magic provided by NeoRV32OnPynq, and how to use it. 
* Introduce the NeoRV32OnPynq overlay GUI

# Introducing the Compiler Magic


This section will introduce the compiler magic provided by NeoRV32OnPynq, riscvc.
This magic enables C programs to be written in any Jupiter Notebook cell, and then compiled simply by running said cell
Before this magic can be used it must be loaded, which is done by the line below

In [1]:
%load_ext NeoRV32OnPynq

The compiler magic is called using <code> %%riscvc <i>progam_name</i> </code>. 
The compiled program can be found in "programs/<i>progam_name</i>/<i>progam_name</i>.bin", where "programs" is a folder in the same location as the notebook in which the compiler magic was run. 

The "programs" folder, and its children (each named for the program they contain), serve as a way to  prevent different programs from inferring with each other. Each child folder contains not only the compiled program (.bin), but also any intermedate files (.c, .o), and some files useful for debugging (.asm, .coe).

<div class="alert alert-box alert-info">
There is no need to worry about creating a "programs" folder, or any of its children, as the compiler magic handles this as needed.
</div>

<div class="alert alert-box alert-info">
The compiler magic is a cell magic, meaning that it needs to be the first statement in a cell, and the rest of the cell's content will be interpreted by the magic.
</div>

## Compiler Magic Example

Below is an example of how to use the compiler magic.
Upon running the cell below a number of effects should be seen:
* If there wasn't already a "programs" folder, one will have been created
* If there wasn't already a "programs\flash_LEDs" folder, one will have been created
* A makefile was added to "programs\flash_LEDs"
* "programs\flash_LEDs\flash_LEDs.c" was (re)created and contains everything but the first line of the compiler magic cell 
* "programs\flash_LEDs\flash_LEDs.c.o" was (re)created
* "programs\flash_LEDs\flash_LEDs.bin" was (re)created
* "programs\flash_LEDs\flash_LEDs.asm" was (re)created
* "programs\flash_LEDs\flash_LEDs.coe" was (re)created


In [2]:
%%riscvc flash_LEDs

//Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;
    
    // Set the LEDs in off state
    LEDs[value] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        LEDs[value] = ~LEDs[value];
    }

    return 0;
}



mkdir -p programs/flash_LEDs

cp /usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/makefile programs/flash_LEDs/makefile

make clean exe NEORV32_HOME=/usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/NEORV_lib LD_SCRIPT=/usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/NeoRV32OnPynq.ld
Memory utilization:
   text	   data	    bss	    dec	    hex	filename
    412	      0	      0	    412	    19c	main.elf
Compiling /usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/NEORV_lib/sw/image_gen/image_gen
Executable (neorv32_exe.bin) size in bytes:
424

mv main.bin flash_LEDs.bin

mv main.asm flash_LEDs.asm

bin2coe -i flash_LEDs.bin -o flash_LEDs.coe -w 32

rm main.* neorv32_exe.bin



## Compiler Magic Example 2


A notebook is not restricted to compiling a single program, below is an example of the compiler magic being used to compile another program, the same set of effects should be seen upon running this cell.

In [3]:
%%riscvc LED_counter

//Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define outsets for LED GPIO handler's regmap
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        for(int32_t i=0; i < 16; i++)
        {
            LEDs[value] = i;
        }
    }

    return 0;
}

mkdir -p programs/LED_counter

cp /usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/makefile programs/LED_counter/makefile

make clean exe NEORV32_HOME=/usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/NEORV_lib LD_SCRIPT=/usr/local/lib/python3.6/dist-packages/NeoRV32OnPynq/NeoRV32OnPynq.ld
Memory utilization:
   text	   data	    bss	    dec	    hex	filename
    416	      0	      0	    416	    1a0	main.elf
Executable (neorv32_exe.bin) size in bytes:
428

mv main.bin LED_counter.bin

mv main.asm LED_counter.asm

bin2coe -i LED_counter.bin -o LED_counter.coe -w 32

rm main.* neorv32_exe.bin



# Interacting with the Processor


Now that we have some programs compiled with the compiler magic, lets look at how to interact with the the NeoRV processor.
All interactions with the processor are done through the `NeoRV32` class within the `NeoRV32OnPynq` module.
The code below first imports the `NeoRV32` class and then used it to setup the board.

In [4]:
from NeoRV32OnPynq import NeoRV32
neorv = NeoRV32()

The recommended method of interacting with the processor is via the GUI provided; which consists of 3  parts, each handling a different stage/type of interaction,
Each part will be covered in their own section, but below is an overview of each.
* Program loading, this section handles the selection and loading of programs to run on the processor
* Execution control, this section controls when the processor can executes instruction, providing stepping, continous, and run until condition options
* Internal Inspection, this sectio display a range of signals internal to the NeoRV32, aiding in debugging and/or tracking of the instructions though the processor

## Program Loading


The program loading section of the GUI is responsbile for enabling a user to select any of the programs found in "programs" and loading them to be run by the NeoRV32. 
It is accessed via the `load_program` function of the `NeoRV32` class as shown below.

In [5]:
neorv.load_program()

To load a program to the process
<ol>
<li>Select the program from the dropdown. If the program you are looking isn't listed in the dropdown, try rerunning `reorv.load_program()` this will check the "programs" folder again and update the program list</li>
<li>Once the desired program is selected in the dropdown, click the "Load Program" button, This will read that programs .bin file, load it to the processor, and reset the processer's PC back to the start of the new program</li>
<li>Once the program has be loaded the "Last Loaded" field will update to show the name of the preogram loaded, and ready to ready to run.</li>
</ol>

## Execution Control

The execution control section of the GUI is responsbile for controling when the process can execute instructions. 
It enables control from the NeoRV32 as varied as running in single clock cycle steps to running the overlay until a specific position within the program is reached.
It is accessed via the `execution_control` function of the `NeoRV32` class as shown below.

In [6]:
 neorv.execution_control()

## show_internals GUI

display a range of signals internal to the overlay, to aid in debugging and/or tracking of the instructions though the overlay

In [7]:
neorv.show_internals()